# Environmental & Social Risk Screening and ESG Action Plan Tracking for a Bank Loan Portfolio
# Project Objective

Simulate how a bank:

-Screens loan applications for E&S risk

-Identifies high-risk clients/vendors

-Tracks ESAPs

-Reports ESG metrics for management / DFIs


In [2]:
import pandas as pd
import numpy as np

np.random.seed(42)

n = 200

data = {
    "Client_ID": [f"C{i:03d}" for i in range(1, n+1)],
    "Sector": np.random.choice(
        ["Agribusiness", "Manufacturing", "Energy", "Construction", "Services"],
        size=n
    ),
    "Country": np.random.choice(
        ["Kenya", "Uganda", "Tanzania", "Rwanda", "DRC"],
        size=n
    ),
    "Loan_Amount_USD": np.random.randint(50000, 5000000, size=n),
    "Environmental_Risk": np.random.choice(["Low", "Medium", "High"], size=n, p=[0.4, 0.4, 0.2]),
    "Social_Risk": np.random.choice(["Low", "Medium", "High"], size=n, p=[0.5, 0.3, 0.2]),
    "HSE_Compliance": np.random.choice(["Yes", "No"], size=n, p=[0.75, 0.25]),
    "ESAP_Status": np.random.choice(
        ["Not Started", "In Progress", "Closed"],
        size=n,
        p=[0.3, 0.4, 0.3]
    ),
    "Green_Finance_Tag": np.random.choice(["Yes", "No"], size=n, p=[0.25, 0.75])
}

df = pd.DataFrame(data)
df.head()


,Client_ID,Sector,Country,Loan_Amount_USD,Environmental_Risk,Social_Risk,HSE_Compliance,ESAP_Status,Green_Finance_Tag
0,C001,Construction,Uganda,2214367,Medium,Low,Yes,Not Started,Yes
1,C002,Services,Tanzania,1954050,Low,Medium,No,Not Started,Yes
2,C003,Energy,Kenya,194356,High,Low,Yes,Closed,No
3,C004,Services,Kenya,4448165,High,Low,Yes,In Progress,No
4,C005,Services,Rwanda,3149320,Medium,Medium,Yes,Not Started,Yes


In [3]:
risk_map = {"Low": 1, "Medium": 2, "High": 3}

df["E_Risk_Score"] = df["Environmental_Risk"].map(risk_map)
df["S_Risk_Score"] = df["Social_Risk"].map(risk_map)

df["Total_ESG_Risk_Score"] = df["E_Risk_Score"] + df["S_Risk_Score"]

df["High_Risk_Flag"] = np.where(df["Total_ESG_Risk_Score"] >= 5, "Yes", "No")

df[["Client_ID", "Environmental_Risk", "Social_Risk", "Total_ESG_Risk_Score", "High_Risk_Flag"]].head()


,Client_ID,Environmental_Risk,Social_Risk,Total_ESG_Risk_Score,High_Risk_Flag
0,C001,Medium,Low,3,No
1,C002,Low,Medium,3,No
2,C003,High,Low,4,No
3,C004,High,Low,4,No
4,C005,Medium,Medium,4,No


In [4]:
portfolio_summary = (
    df.groupby("Sector")
    .agg(
        Total_Loan_Amount=("Loan_Amount_USD", "sum"),
        Avg_ESG_Risk=("Total_ESG_Risk_Score", "mean"),
        High_Risk_Clients=("High_Risk_Flag", lambda x: (x == "Yes").sum())
    )
    .reset_index()
)

portfolio_summary


,Sector,Total_Loan_Amount,Avg_ESG_Risk,High_Risk_Clients
0,Agribusiness,126603595,3.488372,9
1,Construction,126996292,3.326531,5
2,Energy,100996397,3.351351,4
3,Manufacturing,89575129,3.542857,4
4,Services,88706832,3.583333,6


In [5]:
esap_status = (
    df.groupby("ESAP_Status")
    .size()
    .reset_index(name="Number_of_Clients")
)

esap_status


,ESAP_Status,Number_of_Clients
0,Closed,62
1,In Progress,76
2,Not Started,62


In [6]:
green_finance = (
    df.groupby("Green_Finance_Tag")
    .agg(
        Clients=("Client_ID", "count"),
        Loan_Book=("Loan_Amount_USD", "sum")
    )
    .reset_index()
)

green_finance


,Green_Finance_Tag,Clients,Loan_Book
0,No,147,384409077
1,Yes,53,148469168


In [7]:
df.to_csv("synthetic_esg_loan_portfolio.csv", index=False)
portfolio_summary.to_csv("portfolio_esg_summary.csv", index=False)
